In [1]:
import pandas as pd
import tensorflow as tf
from dateutil.parser import parse
dateparse=lambda dates:parse(dates)

from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import numpy as np
from statistics import mean

import IPython
import IPython.display

from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv ('Data/weekly_features.csv')
df = df.drop (columns = ['Unnamed: 0','USD_PHP Historical Data.csv'])
dates = df.year*100+df.week
df['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
mask = (df['Date'] >'1990-09-30') & (df['Date'] <= '2021-09-30')
df= df.loc[mask]
df = df.fillna(method='ffill')

def convert_to_timestamp(x):
    """Convert date objects to integers"""
    return time.mktime(x.to_datetime().timetuple())

# https://www.aiproblog.com/index.php/2018/08/21/4-common-machine-learning-data-transforms-for-time-series-forecasting/
# difference dataset
diff_df=df.drop(columns=['Date', 'year', 'week'])
diff_df = diff_df.diff()
diff_df = diff_df.iloc[1:]
#diff_df['year']=df.year[1:]
diff_df['week']=df.week[1:]
#diff_df['Date']=df.Date[1:]
#diff_df['Date'] = pd.to_datetime(diff_df['Date'])
# convert date to timestamp
#diff_df['Date'] = diff_df['Date'].map(pd.Timestamp.timestamp)

#split the data into training and testing dataset
column_indices = {name: i for i, name in enumerate(diff_df.columns)}

n = len(diff_df)
train_df = diff_df[0:int(n*0.7)]
test_df = diff_df[int(n*0.7):]

num_features = diff_df.shape[1]

#Normalize the data
from sklearn.preprocessing import MinMaxScaler
#col_list = [i for i in diff_df.columns if i != 'Date']
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_df)
scaled_test = scaler.transform(test_df)

In [13]:
train_df = pd.DataFrame(data = scaled_train, columns = ['tmax','tmin','prcp','Crude Oil WTI', 'Gold','Oats','Corn','Soybeans','Wheat','USD_CAD','USD_CNY','USD_EUR','USD_MXN','SWE','SNOW','SNWD','year','week'])
test_df = pd.DataFrame(data = scaled_test, columns = ['tmax','tmin','prcp','Crude Oil WTI', 'Gold','Oats','Corn','Soybeans','Wheat','USD_CAD','USD_CNY','USD_EUR','USD_MXN','SWE','SNOW','SNWD','year', 'week'])

In [14]:
diff_df

,tmax,tmin,prcp,Crude Oil WTI,Gold,Oats,Corn,Soybeans,Wheat,USD_CAD,USD_CNY,USD_EUR,USD_MXN,SWE,SNOW,SNWD,year,week
36,0.231719,-2.491029,-17.361429,-2.800000,-20.660000,0.0500,-5.40,-14.700,-11.00,0.01622,0.00000,-0.00740,0.00600,40.771429,97.000000,1.537537,1990,41
37,-3.653096,-3.812763,1.347143,-5.612000,2.880000,1.6500,2.70,-4.900,0.15,0.00418,0.00000,0.00004,0.00800,74.728571,-488.571429,0.181753,1990,42
38,2.996899,1.217759,-13.064286,3.506000,5.940000,-2.5000,-0.90,-14.950,-0.50,-0.00392,0.00000,0.00008,0.00200,139.442857,11478.000000,1.356229,1990,43
39,-12.418266,-6.051888,23.538571,-0.860000,6.760000,-0.5500,1.30,-14.450,3.10,-0.00002,0.00000,-0.01114,0.00200,254.800000,15332.571429,6.229240,1990,44
40,7.199344,1.608928,-27.015714,-2.454000,-4.120000,-9.0000,-4.65,-15.000,-9.90,-0.00130,0.00000,-0.00472,0.00400,331.885714,-609.571429,-0.468038,1990,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,2.259412,0.171195,-5.414286,2.316000,17.760000,14.5000,-5.80,-3.250,-16.25,-0.00746,-0.01200,-0.00356,0.16694,0.000000,-7.285714,0.000000,2021,34
1674,-4.788346,-3.675091,11.850000,1.814000,14.000000,-8.3000,-26.10,-52.050,-11.55,-0.00446,-0.01714,-0.00548,-0.24268,0.000000,0.000000,0.000000,2021,35
1675,1.073948,-2.842894,-24.675714,0.926667,-13.903333,-13.5625,-19.40,-15.624,-9.50,0.00632,-0.00328,0.00050,-0.09716,0.000000,0.000000,0.000000,2021,36
1676,-1.637910,0.823106,12.104286,0.731333,-23.576667,35.9125,14.60,11.624,1.20,0.00400,-0.00672,0.00346,0.00142,0.000000,0.000000,0.000000,2021,37


#### raw values generate from df

In [9]:
raw_values = df.drop(columns=['Date'])
#raw_values['Date'] = pd.to_datetime(raw_values['Date'])
# convert date to timestamp
#raw_values['Date'] = raw_values['Date'].map(pd.Timestamp.timestamp)

### sliding window

In [3]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

### compile and fit function

In [5]:
# Function of training models
MAX_EPOCHS = 100
EVALUATION_INTERVAL = 200
batch_size = 32
buffer_size = 150
def compile_and_fit(model, train, val, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min') 

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.losses.MeanSquaredError()])

    history = model.fit(train, batch_size=batch_size, epochs=MAX_EPOCHS,
                      validation_data=val,
                        steps_per_epoch=EVALUATION_INTERVAL,
                      callbacks=[early_stopping],
                    validation_steps=10
                       )
    model.reset_states()
    return history

In [6]:
# make a one-step forecast
def forecast_lstm(model, batch_size, X, past_history):
    X = X.reshape(1, past_history, num_features)
    yhat = model.predict(X)
    return yhat

In [7]:
def inverse_difference(history, yhat, position=1):
    return yhat + history[position]

### set up window

In [6]:
past_history = 20
future_target = 5
STEP = 1
X_multi, y_multi = multivariate_data(scaled_train, scaled_train, 0,
                                                   None, past_history,
                                                   future_target, STEP,
                                                   single_step=False)

X_test_multi, y_test_multi = multivariate_data(scaled_test, scaled_test, 0,
                                                   None, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
test_data_multi= tf.data.Dataset.from_tensor_slices((X_test_multi, y_test_multi))
test_data_multi = test_data_multi.batch(batch_size).repeat()

multi_val_performance = {}
multi_test_performance = {}

### split data into multiple training and validation set

In [9]:
#https://www.kaggle.com/code/tomwarrens/timeseriessplit-how-to-use-it/notebook 
splits = TimeSeriesSplit(n_splits=5)
for fold, (train_index, test_index) in enumerate(splits.split(X_multi)):
    print("Fold: {}".format(fold))
    X_train, X_val = X_multi[train_index], X_multi[test_index]
    #X_train, X_val = scaled_train[train_index], scaled_train[test_index]
    #print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
    print('Observations: %d' % (len(X_train) + len(X_val)))
    print('Training Observations: %d' % (len(X_train)))
    print('Testing Observations: %d' % (len(X_val)))
    print("\n")
    
    #y_train, y_test = y[train_index], y[test_index]

Fold: 0
Observations: 376
Training Observations: 189
Testing Observations: 187


Fold: 1
Observations: 563
Training Observations: 376
Testing Observations: 187


Fold: 2
Observations: 750
Training Observations: 563
Testing Observations: 187


Fold: 3
Observations: 937
Training Observations: 750
Testing Observations: 187


Fold: 4
Observations: 1124
Training Observations: 937
Testing Observations: 187




### Baseline

In [13]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, future_target, 1])

baseline = MultiStepLastBaseline()

In [14]:
evaluation_val_list = {}
for fold, (train_index, test_index) in enumerate(splits.split(X_multi)):
    print("Fold: {}".format(fold))
    x_train_multi, x_val_multi = X_multi[train_index], X_multi[test_index]
    y_train_multi, y_val_multi = y_multi[train_index], y_multi[test_index]
    print('Observations: %d' % (len(x_train_multi) + len(x_val_multi)))
    print('Training Observations: %d' % (len(x_train_multi)))
    print('Testing Observations: %d' % (len(x_val_multi)))
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    train_data_multi = train_data_multi.cache().shuffle(buffer_size).batch(batch_size).repeat()

    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
    val_data_multi = val_data_multi.batch(batch_size).repeat()
    
    baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanSquaredError()])
    print(y_train_multi.shape)
    evaluation_val_list ["Fold {}".format(fold)] = baseline.evaluate(val_data_multi, steps=50)



IPython.display.clear_output()
mae = []
# calculate the mean mae
for v in evaluation_val_list.values():
    mae.append(v[1])
Mean_mae = mean(mae)
multi_val_performance['Baseline'] = Mean_mae
multi_test_performance['Baseline'] = baseline.evaluate(test_data_multi, verbose=0, steps=50)

### ARIMA

In [12]:
import numpy as np
import pmdarima as pm
from pmdarima import model_selection
arima_mse = []
for i in range(num_features):
    model1 = pm.ARIMA(order=(2, 1, 0))
    #model2 = pm.ARIMA(order=(1, 1, 2),
                  #seasonal_order=(0, 1, 1, 12),
                  #suppress_warnings=True)
    cv = model_selection.SlidingWindowForecastCV(window_size=20, step=1, h=5)
    model1_cv_scores = model_selection.cross_val_score(
    model1, scaled_test[:468,i], scoring='mean_squared_error', cv=cv, verbose=2)
    m1_average_error = np.average(model1_cv_scores)
    arima_mse.append(m1_average_error)
IPython.display.clear_output()

### LSTM

In [93]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(64, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(future_target*num_features, kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([future_target, num_features])
])

In [94]:
evaluation_val_list = {}
for fold, (train_index, test_index) in enumerate(splits.split(X_multi)):
    print("Fold: {}".format(fold))
    x_train_multi, x_val_multi = X_multi[train_index], X_multi[test_index]
    y_train_multi, y_val_multi = y_multi[train_index], y_multi[test_index]
    print('Observations: %d' % (len(x_train_multi) + len(x_val_multi)))
    print('Training Observations: %d' % (len(x_train_multi)))
    print('Testing Observations: %d' % (len(x_val_multi)))
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    train_data_multi = train_data_multi.cache().shuffle(buffer_size).batch(batch_size).repeat()

    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
    val_data_multi = val_data_multi.batch(batch_size).repeat()
    
    history = compile_and_fit(multi_lstm_model, train_data_multi, val_data_multi)

    evaluation_val_list ["Fold {}".format(fold)] = multi_lstm_model.evaluate(val_data_multi, steps=50)



IPython.display.clear_output()
mae = []
# calculate the mean mae
for v in evaluation_val_list.values():
    mae.append(v[1])
Mean_mae = mean(mae)
multi_val_performance['LSTM'] = Mean_mae
multi_test_performance['LSTM'] = multi_lstm_model.evaluate(test_data_multi, verbose=0, steps=50)

### CNN

In [18]:
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, dense_units]
    #tf.keras.layers.Dense(64, activation='relu'),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(64, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(future_target*num_features,kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([future_target, num_features])
])


In [19]:
evaluation_val_list = {}
for fold, (train_index, test_index) in enumerate(splits.split(X_multi)):
    print("Fold: {}".format(fold))
    x_train_multi, x_val_multi = X_multi[train_index], X_multi[test_index]
    y_train_multi, y_val_multi = y_multi[train_index], y_multi[test_index]
    print('Observations: %d' % (len(x_train_multi) + len(x_val_multi)))
    print('Training Observations: %d' % (len(x_train_multi)))
    print('Testing Observations: %d' % (len(x_val_multi)))
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    train_data_multi = train_data_multi.cache().shuffle(buffer_size).batch(batch_size).repeat()

    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
    val_data_multi = val_data_multi.batch(batch_size).repeat()
    
    history = compile_and_fit(multi_conv_model, train_data_multi, val_data_multi)

    evaluation_val_list ["Fold {}".format(fold)] = multi_conv_model.evaluate(val_data_multi, steps=50)



IPython.display.clear_output()
mae = []
# calculate the mean mae
for v in evaluation_val_list.values():
    mae.append(v[1])
Mean_mae = mean(mae)
multi_val_performance['CNN'] = Mean_mae
multi_test_performance['CNN'] = multi_conv_model.evaluate(test_data_multi, verbose=0, steps=50)

### LSTM-CNN

In [10]:
multi_lstm_cnn = tf.keras.Sequential([
    tf.keras.layers.LSTM(224,input_shape= (past_history,num_features), return_sequences=True),
    tf.keras.layers.Dropout(rate = 0.55),
    tf.keras.layers.LSTM(96,input_shape= (past_history,num_features), return_sequences=True),
    tf.keras.layers.Dropout(rate = 0.15),
    tf.keras.layers.Conv1D(filters=224, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=1),
    tf.keras.layers.Dropout(rate = 0.65),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(448, activation='relu'),
    tf.keras.layers.Dense(future_target*num_features, kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([future_target, num_features]) 
])

In [11]:
evaluation_val_list = {}
for fold, (train_index, test_index) in enumerate(splits.split(X_multi)):
    print("Fold: {}".format(fold))
    x_train_multi, x_val_multi = X_multi[train_index], X_multi[test_index]
    y_train_multi, y_val_multi = y_multi[train_index], y_multi[test_index]
    print('Observations: %d' % (len(x_train_multi) + len(x_val_multi)))
    print('Training Observations: %d' % (len(x_train_multi)))
    print('Testing Observations: %d' % (len(x_val_multi)))
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    train_data_multi = train_data_multi.cache().shuffle(buffer_size).batch(batch_size).repeat()

    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
    val_data_multi = val_data_multi.batch(batch_size).repeat()
    
    history = compile_and_fit(multi_lstm_cnn, train_data_multi, val_data_multi)

    evaluation_val_list ["Fold {}".format(fold)] = multi_lstm_cnn.evaluate(val_data_multi, steps=50)



IPython.display.clear_output()
mae = []
# calculate the mean mae
for v in evaluation_val_list.values():
    mae.append(v[1])
Mean_mae = mean(mae)
multi_val_performance['LSTM-CNN'] = Mean_mae

multi_lstm_cnn.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.losses.MeanSquaredError()])
history = multi_lstm_cnn.fit(X_multi, y_multi, batch_size=batch_size, epochs=MAX_EPOCHS,
                    validation_steps=10
                       )
multi_test_performance['LSTM-CNN'] = multi_lstm_cnn.evaluate(test_data_multi, verbose=0, steps=50)

Epoch 1/100
36/36 [==============================] - 10s 59ms/step - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 2/100
36/36 [==============================] - 2s 58ms/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 3/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0061 - mean_squared_error: 0.0062
Epoch 4/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 5/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 6/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 7/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 8/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 9/100
36/36 [==============================] - 2s 58ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 10/